In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
from fastai.vision.all import *

In [13]:
def es_correcta(imagen): return imagen[0] == 'B'

In [42]:
import skimage
import torchvision.transforms as transforms



# TODO : Revisar class Rotate en la documentacion de fastai.
path = Path('/home/yani/TFG-Dataset/DatasetConIncorrectas/Tipo2/Completo')
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, bs=10, seed=42,
    label_func=es_correcta, item_tfms=[Resize((50,100)) , Rotate()]
)

dls.show_batch()

TypeError: __init__() got an unexpected keyword argument 'rotate'